## read result files
These files have been generated with the [reference coreference scorer script](https://github.com/conll/reference-coreference-scorers).

In [11]:
import pandas as pd
import os 
import re
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.1f}'.format

In [12]:
def extract_totals(fname):
    sout = []
    copy = True
    with open(fname) as fin:
        for line in fin.readlines():
            if line.startswith('====== TOTALS ======='):
                copy = True

            if copy:
                sout.append(line)

            if line.startswith('METRIC'):
                copy = False
    return sout

In [13]:
MENTION_RESULTS_REGEX = re.compile(
    r".*Identification of Mentions: Recall: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*Precision: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*F1: ([0-9.]+)%.*", re.DOTALL)

COREF_RESULTS_REGEX = re.compile(
    r".*(?:Coreference|BLANC): Recall: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*Precision: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*F1: ([0-9.]+)%.*", re.DOTALL)

METRIC_REGEX = re.compile(
    r".*METRIC (.*):.*")



def get_results_dict(lines):
    results = {}

    metric = ''
    for s in lines:
        metric_match = re.match(METRIC_REGEX, s)
        if(metric_match):
            metric, = metric_match.groups()
            results[metric] = {}

        mention_match = re.match(MENTION_RESULTS_REGEX, s)
        if(mention_match):
            r, p, f1 = mention_match.groups()
            results[metric]['mention_rec'] = r
            results[metric]['mention_prec'] = p
            results[metric]['mention_f1'] = f1

        coref_match = re.match(COREF_RESULTS_REGEX, s)
        if(coref_match):
            r, p, f1 = coref_match.groups()
            results[metric]['coref_rec'] = r
            results[metric]['coref_prec'] = p
            results[metric]['coref_f1'] = f1
            
    return results

In [14]:
basepath = '/home/dafne/shared/FilterBubble/coref-resolution/pipeline/results'
os.listdir(basepath)

['dev_coref.txt',
 'dev_e2e-bertje-w2v.txt',
 'dev_e2e.txt',
 'dev_entity.txt',
 'news_coref.txt',
 'news_e2e-bertje-w2v.txt',
 'news_e2e.txt',
 'news_entity.txt',
 'news_goldmentions_coref.txt',
 'news_goldmentions_e2e.txt',
 'news_goldmentions_entity.txt',
 'news_goldparsementions_coref.txt',
 'news_goldparsementions_entity.txt']

In [15]:
results = {}
for fname in os.listdir(basepath):
    expname = fname.split('.')[0]#.split('_')[-1]
    expname = expname.split('_')
    dataset = expname[0]
    tool = expname[-1]
    pipeline = expname[1] if len(expname)>2 else 'endtoend'
    sout = extract_totals(os.path.join(basepath, fname))
    results[(dataset, pipeline, tool)] = get_results_dict(sout)

In [16]:
for exp in results:
    if 'ceafe' in results[exp]:
        results[exp]['conll'] = {}
        for m in ['coref_rec', 'coref_prec','coref_f1']:
            results[exp]['conll'][m] = (float(results[exp]['muc'][m]) + 
                                                 float(results[exp]['bcub'][m]) + 
                                                 float(results[exp]['ceafe'][m]))/3

In [17]:
df_results = pd.DataFrame.from_dict({(*outerKey, innerKey): values for outerKey, innerDict in results.items() 
          for innerKey, values in innerDict.items()}, orient='index')
for c in df_results.columns:
    df_results[c] = df_results[c].astype(float)
    

df_results = df_results.sort_index()
df_results

mention_rec  mention_prec  \
dev  endtoend          coref          bcub          81.5          81.3   
                                      blanc         81.5          81.3   
                                      ceafe         81.5          81.3   
                                      ceafm         81.5          81.3   
                                      conll          nan           nan   
                                      muc           81.5          81.3   
                       e2e            bcub          88.4          88.2   
                                      blanc         88.4          88.2   
                                      ceafe         88.4          88.2   
                                      ceafm         88.4          88.2   
                                      conll          nan           nan   
                                      muc           88.4          88.2   
                       e2e-bertje-w2v bcub          88.3          86.9   
                                      blanc         88.3          86.9   
                                      ceafe         88.3          86.9   
                                      ceafm         88.3          86.9   
                                      conll          nan           nan   
                                      muc           88.3          86.9   
                       entity         bcub          81.8          81.4   
                                      blanc         81.8          81.4   
                                      ceafe         81.8          81.4   
                                      ceafm         81.8          81.4   
                                      conll          nan           nan   
                                      muc           81.8          81.4   
news endtoend          coref          bcub          83.6          86.1   
                                      blanc         83.6          86.1   
                                      ceafe         83.6          86.1   
                                      ceafm         83.6          86.1   
                                      conll          nan           nan   
                                      muc           83.6          86.1   
                       e2e            bcub          91.1          91.6   
                                      blanc         91.1          91.6   
                                      ceafe         91.1          91.6   
                                      ceafm         91.1          91.6   
                                      conll          nan           nan   
                                      muc           91.1          91.6   
                       e2e-bertje-w2v bcub          91.6          91.5   
                                      blanc         91.6          91.5   
                                      ceafe         91.6          91.5   
                                      ceafm         91.6          91.5   
                                      conll          nan           nan   
                                      muc           91.6          91.5   
                       entity         bcub          84.0          86.2   
                                      blanc         84.0          86.2   
                                      ceafe         84.0          86.2   
                                      ceafm         84.0          86.2   
                                      conll          nan           nan   
                                      muc           84.0          86.2   
     goldmentions      coref          bcub          86.6          89.3   
                                      blanc         86.6          89.3   
                                      ceafe         86.6          89.3   
                                      ceafm         86.6          89.3   
                                      conll          nan           nan   
                                      muc        

In [18]:
df_results_swapped = df_results.swaplevel().sort_index()

In [19]:
# Show mention identication scores, sorted on f1
mention_scores = df_results_swapped.loc[(slice(None), slice(None),'muc',slice(None)), ['mention_rec', 'mention_prec', 'mention_f1']]
mention_scores = mention_scores.sort_values('mention_f1', ascending=False)
mention_scores = mention_scores.sort_index(level=[0,1], sort_remaining=False)
mention_scores

mention_rec  mention_prec  \
dev  endtoend          muc e2e                    88.4          88.2   
                           e2e-bertje-w2v         88.3          86.9   
                           entity                 81.8          81.4   
                           coref                  81.5          81.3   
news endtoend          muc e2e-bertje-w2v         91.6          91.5   
                           e2e                    91.1          91.6   
                           entity                 84.0          86.2   
                           coref                  83.6          86.1   
     goldmentions      muc e2e                    91.2         100.0   
                           coref                  86.6          89.3   
                           entity                 86.6          89.3   
     goldparsementions muc coref                  94.5          95.8   
                           entity                 94.5          95.8   

                                           mention_f1  
dev  endtoend          muc e2e                   88.3  
                           e2e-bertje-w2v        87.6  
                           entity                81.6  
                           coref                 81.4  
news endtoend          muc e2e-bertje-w2v        91.5  
                           e2e                   91.3  
                           entity                85.0  
                           coref                 84.8  
     goldmentions      muc e2e                   95.4  
                           coref                 88.0  
                           entity                88.0  
     goldparsementions muc coref                 95.2  
                           entity                95.2

In [20]:
sorted_scores = df_results_swapped.sort_values('coref_f1',ascending=False)
sorted_scores = sorted_scores.sort_index(level=[0,1,2], sort_remaining=False)[['coref_rec', 'coref_prec', 'coref_f1']]
sorted_scores

coref_rec  coref_prec  coref_f1
dev  endtoend          bcub  e2e-bertje-w2v       73.7        80.3      76.8
                             e2e                  71.9        81.8      76.5
                             entity               65.3        70.8      68.0
                             coref                57.9        62.2      60.0
                       blanc e2e                  61.9        72.5      66.2
                             e2e-bertje-w2v       62.2        69.2      65.1
                             entity               49.7        44.5      46.6
                             coref                34.2        34.3      34.2
                       ceafe e2e                  80.3        69.9      74.7
                             e2e-bertje-w2v       79.2        70.1      74.3
                             entity               68.8        63.0      65.8
                             coref                57.3        54.8      56.0
                       ceafm e2e                  72.6        72.5      72.6
                             e2e-bertje-w2v       73.2        72.0      72.6
                             entity               63.5        63.1      63.3
                             coref                50.1        50.0      50.1
                       conll e2e-bertje-w2v       68.5        74.5      70.8
                             e2e                  67.2        75.6      70.3
                             entity               59.3        63.3      61.0
                             coref                41.1        42.0      41.5
                       muc   e2e-bertje-w2v       52.5        73.2      61.2
                             e2e                  49.4        75.1      59.6
                             entity               43.7        56.1      49.2
                             coref                 8.0         9.1       8.5
news endtoend          bcub  e2e-bertje-w2v       78.9        88.0      83.2
                             e2e                  78.3        87.4      82.6
                             entity               69.3        79.5      74.1
                             coref                65.8        71.8      68.7
                       blanc e2e-bertje-w2v       62.9        83.4      69.8
                             e2e                  61.9        78.7      68.0
                             entity               51.9        61.3      55.9
                             coref                45.1        51.2      47.8
                       ceafe e2e-bertje-w2v       87.1        77.2      81.9
                             e2e                  86.6        77.2      81.6
                             entity               75.5        68.7      71.9
                             coref                68.0        65.8      66.9
                       ceafm e2e-bertje-w2v       80.2        80.1      80.2
                             e2e                  79.3        79.7      79.5
                             entity               69.4        71.2      70.3
                             coref                62.6        64.5      63.5
                       conll e2e-bertje-w2v       72.8        82.0      76.2
                             e2e                  72.2        81.9      75.8
                             entity               61.9        71.2      65.4
                             coref                53.3        56.8      54.9
                       muc   e2e-bertje-w2v       52.3        80.8      63.5
                             e2e                  51.7        81.0      63.1
                             entity               40.9        65.2      50.3
                             coref                26.2        32.9      29.1
     goldmentions      bcub  e2e                  78.4        94.8      85.8
                             entity               71.6        82.5      76.6
                             coref                71.9        78.2      74.9
                       blanc e2e         